# P2F Python Client Library Example

Welcome to the Client Library Example Notebook.

First, we need to import a few things, the P2F Client library and the p2f_pydantic library. The client library is an implementation of requests and will send and recieve pydantic objects between your computer and the API. 

## Pydantic?

Pydantic is a library that works on the back-end of the API to enforce data types and converting data structures to JSON.  

So why do you need it? So we can have the clients and the server in sync and agree on the definition of data. The library will also encode your local data into JSON for requests to send over the REST API standard in a fairly painless manner. 

In [1]:
from p2f_client.p2f_client import P2F_Client
import p2f_pydantic
import datetime

First, let's initialize a client, this method will probably get updated in the future, but below this is the initialization method. 

In [2]:
client = P2F_Client(hostname="localhost", port=8000, https=False)

## Datasets

Below we're going to start by uploading new datasets to the Portal. What is happening is I am creating a pydantic Datasets object for each of my three datasets. 

After that, there are two methods of uploading, what is shown here is `add_dataset`, which places the dataset into a queue that can then all be uploaded at once with `upload_datasets`. 

In [3]:
# Create Pydantic Objects
dataset1 = p2f_pydantic.datasets.Datasets(doi="10.1594/PANGAEA.920596",
                                          title="Last Glacial Maximum SST proxy collection and data assimilation",
                                          publication_date=datetime.datetime(2020, 7, 21),
                                          is_new_p2f=False,
                                          is_sub_dataset=False)

dataset2 = p2f_pydantic.datasets.Datasets(doi="10.5194/gmd-12-3149-2019-supplement",
                             title="The DeepMIP contribution to PMIP4: methodologies for selection, compilation and analysis of latest Paleocene and early Eocene climate proxy data, incorporating version 0.1 of the DeepMIP database",
                             publication_date=datetime.datetime(2019, 7, 25),
                             is_new_p2f=False,
                             is_sub_dataset=False)

dataset3 = p2f_pydantic.datasets.Datasets(doi="10.1594/PANGAEA.911847", 
                             title="Sea surface temperature anomalies for Pliocene interglacial KM5c (PlioVAR)", 
                             publication_date=datetime.datetime(2020, 2, 7), 
                             is_new_p2f=False, 
                             is_sub_dataset=False)

# Add datasets to Queue
client.datasets.add_dataset(dataset1)
client.datasets.add_dataset(dataset2)
client.datasets.add_dataset(dataset3)

# Upload datasets in the Queue
client.datasets.upload_datasets()

[Datasets(dataset_identifier=UUID('67e92576-6a1a-44a9-9293-c89cd366dde7'), doi='10.1594/PANGAEA.920596', title='Last Glacial Maximum SST proxy collection and data assimilation', sub_dataset_name=None, publication_date=datetime.date(2020, 7, 21), is_new_p2f=False, is_sub_dataset=False),
 Datasets(dataset_identifier=UUID('193626a1-b53b-4d8c-92c2-d586b1bf7235'), doi='10.5194/gmd-12-3149-2019-supplement', title='The DeepMIP contribution to PMIP4: methodologies for selection, compilation and analysis of latest Paleocene and early Eocene climate proxy data, incorporating version 0.1 of the DeepMIP database', sub_dataset_name=None, publication_date=datetime.date(2019, 7, 25), is_new_p2f=False, is_sub_dataset=False),
 Datasets(dataset_identifier=UUID('eb649739-b924-405b-a9dc-052e2e81dba2'), doi='10.1594/PANGAEA.911847', title='Sea surface temperature anomalies for Pliocene interglacial KM5c (PlioVAR)', sub_dataset_name=None, publication_date=datetime.date(2020, 2, 7), is_new_p2f=False, is_sub_

You can also just upload a dataset directly with `upload_dataset()`

In [4]:
dataset4 = p2f_pydantic.datasets.Datasets(doi="10.5194/gmd-12-3149-2019-supplement", 
                                          title="SDF02_Sites.xlsx",
                                          publication_date=datetime.datetime(2019, 7, 25), 
                                          is_new_p2f=False, 
                                          is_sub_dataset=True)
client.datasets.upload_dataset(dataset4)

Datasets(dataset_identifier=UUID('62e64982-c1bf-4f92-84f1-6b170cd8d5c1'), doi='10.5194/gmd-12-3149-2019-supplement', title='SDF02_Sites.xlsx', sub_dataset_name=None, publication_date=datetime.date(2019, 7, 25), is_new_p2f=False, is_sub_dataset=True)

Now that we have datasets uploaded, we can check datasets that exist on the server.

In [5]:
# Get the list of datasets on the server
client.datasets.list_remote_datasets()

[Datasets(dataset_identifier=UUID('67e92576-6a1a-44a9-9293-c89cd366dde7'), doi='10.1594/PANGAEA.920596', title='Last Glacial Maximum SST proxy collection and data assimilation', sub_dataset_name=None, publication_date=datetime.date(2020, 7, 21), is_new_p2f=False, is_sub_dataset=False),
 Datasets(dataset_identifier=UUID('193626a1-b53b-4d8c-92c2-d586b1bf7235'), doi='10.5194/gmd-12-3149-2019-supplement', title='The DeepMIP contribution to PMIP4: methodologies for selection, compilation and analysis of latest Paleocene and early Eocene climate proxy data, incorporating version 0.1 of the DeepMIP database', sub_dataset_name=None, publication_date=datetime.date(2019, 7, 25), is_new_p2f=False, is_sub_dataset=False),
 Datasets(dataset_identifier=UUID('eb649739-b924-405b-a9dc-052e2e81dba2'), doi='10.1594/PANGAEA.911847', title='Sea surface temperature anomalies for Pliocene interglacial KM5c (PlioVAR)', sub_dataset_name=None, publication_date=datetime.date(2020, 2, 7), is_new_p2f=False, is_sub_

You can also delete a dataset, below I am just deleting the last dataset on the server. **Please don't do this in real usage**

In [6]:
client.datasets.delete_remote_dataset(client.datasets.list_remote_datasets()[-1].dataset_identifier)

## Adding a Table

Adding a table is going to be a bit more complex, we need to prepopulate our data types, and then iterate through our table. 

For this example we're going to look at one of the tables (sub-datasets) of the PlioVAR article from above. First let's add our dataset. 

In [7]:
pliovartab = p2f_pydantic.datasets.Datasets(doi="10.1594/PANGAEA.911847", 
                             title="PlioVAR-KM5c_T.tab", 
                             publication_date=datetime.datetime(2020, 2, 7), 
                             is_new_p2f=False, 
                             is_sub_dataset=True)
pliovartab = client.datasets.upload_dataset(pliovartab)

For this dataset we have 20 columns, four of which can be grouped into being a location, and nine that are Sea Surface Temperature related. Let's check the API server for what data types exist for Sea Surface Temperatures. 

In [8]:
client.harm_data_type.list_data_types()

[]

For me, the developer, there are currently no data types on the P2F API server, so I need to create them. 

Reading the documentation on the dataset I know I need to create a data type for each column

In [9]:
# Base SSTs
SST_UK37 = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature", 
                                                       unit_of_measurement="°C",
                                                       method="UK37",
                                                       is_proxy=True)
SST_UK37_BAYSPLINE = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature", 
                                                       unit_of_measurement="°C",
                                                       method="UK37 with Bayspline calibration",
                                                       is_proxy=True)
SST_MgCa = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature", 
                                                       unit_of_measurement="°C",
                                                       method="Ratios Mg/Ca",
                                                       is_proxy=True)
SST_MgCa_BAYMAG = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature", 
                                                       unit_of_measurement="°C",
                                                       method="Ratios Mg/Ca with BAYMAG calibration",
                                                       is_proxy=True)
SST_FORWARD = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature", 
                                                       unit_of_measurement="°C",
                                                       method="Forward-modelled 'core-top' Mg/Ca sea surface temperature from BAYMAG",
                                                       is_proxy=True)

# Anomalous SSTs
SSTA_UK37 = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature Anomaly", 
                                                       unit_of_measurement="°C",
                                                       method="UK37",
                                                       is_proxy=True)
SSTA_UK37_BAYSPLINE = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature Anomaly", 
                                                       unit_of_measurement="°C",
                                                       method="UK37 with Bayspline calibration",
                                                       is_proxy=True)
SSTA_MgCa = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature Anomaly", 
                                                       unit_of_measurement="°C",
                                                       method="Ratios Mg/Ca",
                                                       is_proxy=True)
SSTA_MgCa_BAYMAG = p2f_pydantic.harm_data_types.harm_data_type(measure="Sea Surface Temperature Anomaly", 
                                                       unit_of_measurement="°C",
                                                       method="Ratios Mg/Ca with BAYMAG calibration",
                                                       is_proxy=True)

After we create the data types, we upload them individually to the API so we can get their `datatype_id` back for use with our numerical uploads later. 

In [10]:
SST_UK37 = client.harm_data_type.upload_data_type(SST_UK37)
SSTA_UK37 = client.harm_data_type.upload_data_type(SSTA_UK37)
SST_UK37_BAYSPLINE = client.harm_data_type.upload_data_type(SST_UK37_BAYSPLINE)
SSTA_UK37_BAYSPLINE = client.harm_data_type.upload_data_type(SSTA_UK37_BAYSPLINE)
SST_MgCa = client.harm_data_type.upload_data_type(SST_MgCa)
SSTA_MgCa = client.harm_data_type.upload_data_type(SSTA_MgCa)
SST_MgCa_BAYMAG = client.harm_data_type.upload_data_type(SST_MgCa_BAYMAG)
SSTA_MgCa_BAYMAG = client.harm_data_type.upload_data_type(SSTA_MgCa_BAYMAG)
SST_FORWARD = client.harm_data_type.upload_data_type(SST_FORWARD)

In [11]:
# Confirm upload
client.harm_data_type.list_data_types()

[harm_data_type(datatype_id=UUID('b45626b3-c468-4465-b8c4-54c3aad9cf08'), measure='Sea Surface Temperature', unit_of_measurement='°C', method='UK37', is_proxy=True),
 harm_data_type(datatype_id=UUID('07b6893f-f49d-4844-affe-cea5c1fa2a7f'), measure='Sea Surface Temperature Anomaly', unit_of_measurement='°C', method='UK37', is_proxy=True),
 harm_data_type(datatype_id=UUID('13fc61ce-071a-4172-8df9-055e06b6b332'), measure='Sea Surface Temperature', unit_of_measurement='°C', method='UK37 with Bayspline calibration', is_proxy=True),
 harm_data_type(datatype_id=UUID('65d36b62-ccc0-4a5d-b7cd-e1bc240d9606'), measure='Sea Surface Temperature Anomaly', unit_of_measurement='°C', method='UK37 with Bayspline calibration', is_proxy=True),
 harm_data_type(datatype_id=UUID('532c25e5-d966-4eb4-88a1-0f23259c717a'), measure='Sea Surface Temperature', unit_of_measurement='°C', method='Ratios Mg/Ca', is_proxy=True),
 harm_data_type(datatype_id=UUID('feff6cfc-4ed2-41a2-a6ad-4825bf60747a'), measure='Sea Surfa

Next let's load our dataset into Pandas. 

In [12]:
import pandas as pd

In [13]:
plio_df = pd.read_csv("Pliocene_SSTs/PlioVAR-KM5c_T.tab", skiprows=99, sep="\t")
plio_df

,Event,Site,Latitude,Longitude,Elevation [m a.s.l.],"SST [°C] (for KM5c, Calculated from UK3...)","SST [°C] (for KM5c, Calculated from UK3...).1","SST [°C] (for KM5c, Calculated from Mg/...)","SST [°C] (for KM5c, Calculated from Mg/...).1",Species (Planktonic foraminifera speci...),SST [°C] (Forward-modelled 'core-top' M...),Season (Interpretation of the seasona...),SST anomaly [°C] (for KM5c minus the Pre-indust...),SST anomaly [°C] (for KM5c minus the Pre-indust...).1,SST anomaly [°C] (for KM5c minus the Pre-indust...).2,SST anomaly [°C] (for KM5c minus the Pre-indust...).3,Reference (PANGAEA DOI or reference of d...),Reference (PANGAEA DOI or reference of d...).1,Reference (NCDC DOI or reference of data...),Reference (Other DOI or reference of dat...)
0,162-907,ODP907,69.241500,-12.698300,-1801.5,3.500000,3.159048,NaN,NaN,NaN,NaN,NaN,2.046593,1.705641,NaN,NaN,doi:10.1594/PANGAEA.877308,NaN,NaN,doi:10.1038/ngeo2813
1,104-642,ODP642,67.222000,2.926317,-1280.9,13.840000,13.181545,NaN,NaN,NaN,NaN,NaN,7.059326,6.400871,NaN,NaN,NaN,NaN,NaN,doi:10.1016/j.epsl.2016.04.024
2,162-982,ODP982,57.516700,-15.866700,-1134.2,16.968000,16.611689,NaN,NaN,NaN,NaN,NaN,6.299140,5.942828,NaN,NaN,NaN,NaN,NaN,doi:10.1029/2008PA001669
3,341-U1417,U1417,56.959800,-147.109000,-4187.0,10.082234,9.496817,NaN,NaN,NaN,NaN,NaN,2.867241,2.281824,NaN,NaN,doi:10.1594/PANGAEA.899064,NaN,NaN,NaN
4,94-609,DSDP609,49.877783,-24.238117,-3883.0,17.744444,17.052641,11.660000,12.458583,bulloides,13.8771,annual,4.399195,3.707392,-1.685249,-0.886666,doi:10.1594/PANGAEA.271535,NaN,NaN,doi:10.1002/2017pa003101
5,306-U1313,U1313,41.001233,-32.957167,-3412.6,21.598750,20.855745,22.240000,26.990420,bulloides,18.6015,annual,3.835015,3.092010,4.476265,9.226685,doi:10.1594/PANGAEA.804674,doi:10.1594/PANGAEA.744483,NaN,NaN
6,94-607,DSDP607,41.001200,-32.957300,-3427.0,20.575000,20.332318,NaN,NaN,NaN,NaN,NaN,2.811265,2.568583,NaN,NaN,NaN,NaN,https://doi.org/10.25921/phvg-5h05,NaN
7,Punta_Piccola,SicilyPuntaPiccola,37.288890,13.493330,0.0,25.785000,25.571926,NaN,NaN,NaN,NaN,NaN,6.568540,6.355466,NaN,NaN,NaN,NaN,NaN,doi:10.1016/j.epsl.2014.10.006
8,339-U1387,U1387,36.805000,-7.718000,-558.0,26.886364,27.219137,NaN,NaN,NaN,NaN,NaN,8.433762,8.766535,NaN,NaN,NaN,NaN,NaN,doi:10.1016/j.gloplacha.2015.07.001
9,93-603,DSDP603,35.495267,-70.029267,-4633.0,NaN,NaN,20.427273,25.157464,bulloides,24.5493,annual,NaN,NaN,-1.791507,2.938684,doi:10.1594/PANGAEA.804666,doi:10.1594/PANGAEA.804667,NaN,NaN


In [14]:
from pprint import pprint
import numpy as np

Here we will iterate through all of the records using `iterrows()` and then for each row of the table we will add in some information. 

**Note on row hash** - I was trying to create a universal and unique (kind of a UUID) way for a row to be referred to as without just using the row number but also to be recalculable over and over again. This may be a pointless thing to do here, but it is currently the way its implemented. 

The steps as seen below are:

1. Calculate a row hash, and insert the row as a data record that will continue to be referred to by its record_hash.
2. Create (or find an existing) location record and add to the database, then assign location to the data record.
3. Then for each data type, add the numerical records as long as they exist, if they are null, in my opinion it is best not to add in empty data (this can be a point for reconsideration).

In [ ]:
# Get the UUID of the dataset after it was created by the database above
dataset_id = pliovartab.dataset_identifier

# Iterate through the rows using iterrows, ix is index and row is the rest of the content of the row
for ix, row in plio_df.iterrows():
    # Calculate the row hash
    row_hash = client.harm_data_records.calculate_hash(dataset_id=dataset_id,
                                                       row_number=ix,
                                                       debugging=True)
    # Create the data record object for uploading
    row_record = p2f_pydantic.harm_data_record.harm_data_record(fk_dataset=dataset_id,
                                                                record_hash=row_hash)
    # Upload the data record
    client.harm_data_records.upload_data_record(row_record)
    # Create the location object for uploading
    location = p2f_pydantic.harm_data_metadata.harm_location(location_name=row.iloc[1],
                                                             latitude=row.iloc[2],
                                                             longitude=row.iloc[3],
                                                             elevation=row.iloc[4], 
                                                             location_age=0)
    # Upload the location to the database
    location = client.harm_location.upload_harm_location(location) 
    # Assign that newly created location to this data record
    client.harm_location.assign_location_to_record(location.location_identifier, row_hash)

    # Create some lists that we can use indexes on to reuse code
    #     This first one has the data types from above so we can grab
    #      their datatype_id
    col_datatype = [SST_UK37, SST_UK37_BAYSPLINE, SST_MgCa, 
                    SST_MgCa_BAYMAG, SST_FORWARD, SSTA_UK37,
                    SSTA_UK37_BAYSPLINE, SSTA_MgCa, SSTA_MgCa_BAYMAG] 
    #     This one is the column number and this order matches the order
    #      of the data in the above list. 
    cols = [5, 6, 7, 8, 10, 12, 13, 14, 15]
    # Iterate through the column numbers
    for col in cols:
        # Check that the value is not null
        if not np.isnan(row.iloc[col]):
            # Create the numerical insert object for uploading
            new_numerical_insert = p2f_pydantic.harm_data_numerical.insert_harm_numerical(fk_data_record=row_hash,
                                                                                          fk_data_type=col_datatype[cols.index(col)].datatype_id,
                                                                                          numerical_type="FLOAT",
                                                                                          value=row.iloc[col])
            # Upload the numerical object
            client.harm_numerical.upload_harm_numerical(new_numerical_insert)

b80edf67-b8b0-454b-9323-428e53f34098
value=3.5 fk_data_record='aa3613f2567ff980a9e4b02b1b3bc865' fk_data_type=UUID('b45626b3-c468-4465-b8c4-54c3aad9cf08')
value=3.15904776 fk_data_record='aa3613f2567ff980a9e4b02b1b3bc865' fk_data_type=UUID('13fc61ce-071a-4172-8df9-055e06b6b332')
value=2.046592968 fk_data_record='aa3613f2567ff980a9e4b02b1b3bc865' fk_data_type=UUID('07b6893f-f49d-4844-affe-cea5c1fa2a7f')
value=1.705640728 fk_data_record='aa3613f2567ff980a9e4b02b1b3bc865' fk_data_type=UUID('65d36b62-ccc0-4a5d-b7cd-e1bc240d9606')


900c6623-f50a-4928-af85-ef8f0422a859
value=13.84 fk_data_record='8b3cbcae3c405ee06b0824017cb76f73' fk_data_type=UUID('b45626b3-c468-4465-b8c4-54c3aad9cf08')
value=13.18154489 fk_data_record='8b3cbcae3c405ee06b0824017cb76f73' fk_data_type=UUID('13fc61ce-071a-4172-8df9-055e06b6b332')
value=7.059325807 fk_data_record='8b3cbcae3c405ee06b0824017cb76f73' fk_data_type=UUID('07b6893f-f49d-4844-affe-cea5c1fa2a7f')
value=6.400870697 fk_data_record='8b3cbcae3c405ee06b08240